# Combining Variables

## Seeing linear regression in action

In [1]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import scale

boston = load_boston()
X, y = scale(boston.data), boston.target

In [2]:
from sklearn.linear_model import LinearRegression

regression = LinearRegression()
regression.fit(X, y)

print('R2 %0.3f' % regression.score(X, y))

R2 0.741


In [3]:
print([a + ':' + str(round(b, 1)) for a, b in 
       zip(boston.feature_names, regression.coef_)])

['CRIM:-0.9', 'ZN:1.1', 'INDUS:0.1', 'CHAS:0.7', 'NOX:-2.1', 'RM:2.7', 'AGE:0.0', 'DIS:-3.1', 'RAD:2.7', 'TAX:-2.1', 'PTRATIO:-2.1', 'B:0.9', 'LSTAT:-3.7']


# Mixing Variable Types

## Modeling the responses

In [4]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
lbl = LabelEncoder()
enc = OneHotEncoder()
qualitative = ['red', 'red', 'green', 'blue', 
               'red', 'blue', 'blue', 'green']
labels = lbl.fit_transform(qualitative).reshape(8,1)
print(enc.fit_transform(labels).toarray())

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


## Dealing with complex relations

In [5]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection  import train_test_split
from sklearn.metrics import r2_score

pf = PolynomialFeatures(degree=2)
poly_X = pf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(poly_X,
                    y, test_size=0.33, random_state=42)

from sklearn.linear_model import Ridge
reg_regression = Ridge(alpha=0.1, normalize=True)
reg_regression.fit(X_train,y_train)
print ('R2: %0.3f' % r2_score(y_test,reg_regression.predict(X_test)))

R2: 0.819


# Switching to Probabilities

## Specifying a binary response

In [6]:
import numpy as np

a = np.array([0, 0, 0, 0, 1, 1, 1, 1])
b = np.array([1, 2, 3, 4, 5, 6, 7, 8]).reshape(8,1)
from sklearn.linear_model import LinearRegression
regression = LinearRegression()
regression.fit(b,a)
print (regression.predict(b)>0.5)

[False False False False  True  True  True  True]


## Transforming numeric estimates into probabilities

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection  import train_test_split

binary_y = np.array(y >= 40).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, 
            binary_y, test_size=0.33, random_state=5)
logistic = LogisticRegression()
logistic.fit(X_train,y_train)
from sklearn.metrics import accuracy_score
print('In-sample accuracy: %0.3f' % 
      accuracy_score(y_train, logistic.predict(X_train)))
print('Out-of-sample accuracy: %0.3f' % 
      accuracy_score(y_test, logistic.predict(X_test)))

In-sample accuracy: 0.973
Out-of-sample accuracy: 0.958


In [8]:
for var,coef in zip(boston.feature_names,
                    logistic.coef_[0]):
        print ("%7s : %7.3f" %(var, coef)) 

   CRIM :  -0.006
     ZN :   0.197
  INDUS :   0.580
   CHAS :  -0.023
    NOX :  -0.236
     RM :   1.426
    AGE :  -0.048
    DIS :  -0.365
    RAD :   0.645
    TAX :  -0.220
PTRATIO :  -0.554
      B :   0.049
  LSTAT :  -0.803


In [9]:
print('\nclasses:',logistic.classes_)
print('\nProbs:\n',logistic.predict_proba(X_test)[:3,:])


classes: [0 1]

Probs:
 [[0.39022779 0.60977221]
 [0.93856655 0.06143345]
 [0.98425623 0.01574377]]


# Guessing the Right Features

## Defining the outcome of incompatible features

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

X_train, X_test, y_train, y_test = train_test_split(X, 
                y, test_size=0.33, random_state=42)
check = [2**i for i in range(8)]
for i in range(2**7+1):
    X_train = np.column_stack((X_train,np.random.random(
        X_train.shape[0])))
    X_test = np.column_stack((X_test,np.random.random(
        X_test.shape[0])))
    regression.fit(X_train, y_train)
    if i in check:
        print ("Random features: %i -> R2: %0.3f" % 
               (i, r2_score(y_train,regression.predict(X_train))))

Random features: 1 -> R2: 0.743
Random features: 2 -> R2: 0.743
Random features: 4 -> R2: 0.745
Random features: 8 -> R2: 0.747
Random features: 16 -> R2: 0.752
Random features: 32 -> R2: 0.760
Random features: 64 -> R2: 0.788
Random features: 128 -> R2: 0.842


In [11]:
regression.fit(X_train, y_train)
print ('R2 %0.3f' 
   % r2_score(y_test,regression.predict(X_test)))
# Please notice that the R2 result may change from run to 
# run due to the random nature of the experiment

R2 0.491


## Solving overfitting using selection and regularization

In [12]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

pf = PolynomialFeatures(degree=2)
poly_X = pf.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(poly_X,
                    y, test_size=0.33, random_state=42)

from sklearn.linear_model import Ridge
reg_regression = Ridge(alpha=0.1, normalize=True)
reg_regression.fit(X_train,y_train)
print ('R2: %0.3f' 
   % r2_score(y_test,reg_regression.predict(X_test)))

R2: 0.819


# Learning One Example at a Time

## Understanding how SDG is different

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor

X_train, X_test, y_train, y_test = train_test_split(X, 
                    y, test_size=0.33, random_state=42)
SGD = SGDRegressor(penalty=None,
                   learning_rate='invscaling', 
                   eta0=0.01, power_t=0.25,
                   max_iter=5, tol=None)

power = 17
check = [2**i for i in range(power+1)]
for i in range(400):
    for j in range(X_train.shape[0]):
        SGD.partial_fit(X_train[j,:].reshape(1,13), 
                        y_train[j].reshape(1,))
        count = (j+1) + X_train.shape[0] * i
        if count in check:
            R2 = r2_score(y_test,SGD.predict(X_test))
            print ('Example %6i R2 %0.3f coef: %s' % 
            (count, R2, ' '.join(map(lambda x:'%0.3f' %x, SGD.coef_))))

Example      1 R2 -6.255 coef: 0.113 -0.071 0.148 -0.040 0.075 -0.021 0.146 -0.113 0.243 0.224 0.118 0.037 0.110
Example      2 R2 -6.168 coef: 0.066 -0.139 0.087 -0.078 0.055 -0.114 0.254 -0.054 0.154 0.140 0.282 0.068 0.152
Example      4 R2 -6.060 coef: -0.072 -0.195 0.319 -0.171 0.064 -0.206 0.527 0.048 -0.041 0.266 0.075 0.219 0.353
Example      8 R2 -5.776 coef: -0.246 -0.504 0.605 -0.343 0.098 0.005 0.807 -0.304 -0.095 0.332 -0.067 0.399 0.024
Example     16 R2 -5.145 coef: -0.437 -0.430 0.298 -0.571 -0.002 0.004 0.519 -0.423 -0.279 0.292 -0.544 0.665 -0.065
Example     32 R2 -4.495 coef: -0.554 -0.308 0.441 1.224 0.051 0.315 0.387 -0.566 0.055 0.629 -0.367 0.726 -0.513
Example     64 R2 -2.947 coef: -0.974 0.419 0.107 1.648 -0.409 1.687 -0.428 -0.201 -0.029 0.447 -1.246 1.166 -1.914
Example    128 R2 -1.791 coef: -0.533 0.863 0.118 1.137 -0.585 1.823 -0.290 -0.178 -0.283 0.094 -1.984 1.166 -2.030
Example    256 R2 -0.607 coef: -0.783 0.619 -0.177 1.368 -0.771 3.135 -0.305 -0.51